In [1]:
import pandas as pd
import os
import numpy as np

In [38]:
# Đường dẫn thư mục hiện tại
my_url = "."

# Đọc file và gộp các file thành các dataframe 
prefix_path = f"{my_url}/data/data2014-2018/"
dir_list = os.listdir(prefix_path)
diem_df_list = []
khgd_df_list = []
for file_name in dir_list:
    if file_name.startswith("DataPhanCong"):
        # Data phân công giảng dạy
        # Các trường dữ liệu MaMH, DSCBGD, NhomTo thường bị nhận nhầm thành số
        khgd_df_i = pd.read_excel(f"{prefix_path}/{file_name}", index_col=None, converters={"MaMH": str, "DSCBGD": str, "NhomTo": str})
        khgd_df_i = khgd_df_i[["MaMH", "NhomTo", "DSCBGD"]]
        khgd_df_i.insert(1, "NHHK", file_name[13:17] + file_name.split('.')[0].split('-')[-1], allow_duplicates=False)
        khgd_df_list.append(khgd_df_i)
    else:
        # Data điểm
        diem_df_i = pd.read_excel(f"{prefix_path}/{file_name}", index_col=None, converters={"NHHK": str, "NhomTo": str})
        diem_df_list.append(diem_df_i)

# Nối các dataframe và đánh lại index
diem_df = pd.concat(diem_df_list).reset_index(drop=True)
khgd_df = pd.concat(khgd_df_list).reset_index(drop=True)

In [39]:
# Lấy các trường cần thiết
diem_df_temp = diem_df[["MaSV", "MaMH", "NHHK", "NhomTo", "DiemHP", "DatHP"]]
khgd_df_temp = khgd_df[["MaMH", "NHHK", "NhomTo", "DSCBGD"]]

In [40]:
# Drop các record có NaN
diem_df_temp = diem_df_temp.dropna()

In [41]:
diem_df_temp

,MaSV,MaMH,NHHK,NhomTo,DiemHP,DatHP
0,N14DCDT066,BAS1105,20141,03,9.0,1
1,N14DCDT066,BAS1106,20141,05,5.2,1
2,N14DCDT066,BAS1111,20141,03,6.7,1
3,N14DCDT066,BAS1201,20141,13,6.1,1
4,N14DCDT066,BAS1203,20141,13,5.6,1
...,...,...,...,...,...,...
89541,N18DCCN118,INT1461,20212,01,8.6,1
89542,N18DCCN118,INT14103,20221,01,8.2,1
89543,N18DCCN118,INT14104,20221,01,5.3,1
89544,N18DCCN118,INT1450,20221,01,6.8,1


In [42]:
khgd_df_temp

,MaMH,NHHK,NhomTo,DSCBGD
0,BAS1115,20141,08,0211034
1,BAS1115,20141,13,0211034
2,BAS1115,20141,10,0211033
3,BAS1115,20141,06,0211033
4,BAS1115,20141,07,0211023
...,...,...,...,...
12451,ELE1317,20223,01,0211043
12452,ELE1433,20223,01,0211047
12453,BSA1241,20223,01,0211179
12454,SKD1103,20223,01,GV/N-20229


In [43]:
# Lưu tạm thời ra file csv
diem_df_temp.to_csv(f"{my_url}/data/diem.csv")
khgd_df_temp.to_csv(f"{my_url}/data/khgd.csv")

In [44]:
# Đọc dữ liệu từ diem.csv và khgd.csv
diem_df = pd.read_csv(f"{my_url}/data/diem.csv", index_col=0, converters={"MaMH": str, "NHHK": str, "NhomTo": str})
khgd_df = pd.read_csv(f"{my_url}/data/khgd.csv", index_col=0, converters={"MaMH": str, "DSCBGD": str, "NhomTo": str, "NHHK": str})

In [45]:
# Lấy sinh viên CNTT và AT
diem_df = diem_df.loc[diem_df["MaSV"].str.contains("DCCN|DCAT")]
# Đổi tên cột
khgd_df.rename(columns={'DSCBGD': 'MaNV'}, inplace=True)
# Bỏ các record có NhomTo là HL (Học lại)
khgd_df = khgd_df.loc[khgd_df["NhomTo"] != 'HL']

In [46]:
print(f"Số lượng record dữ liệu: {len(diem_df)}")
print(f"Số lượng record môn INT1306: {len(diem_df[diem_df['MaMH'] == 'INT1306'])}")

Số lượng record dữ liệu: 32277
Số lượng record môn INT1306: 1047


In [47]:
diem_df_1306 = diem_df[diem_df["MaMH"] == "INT1306"]
print(f'Số lượng sinh viên học môn INT1306: {len(set(diem_df_1306["MaSV"]))}')

Số lượng sinh viên học môn INT1306: 521


In [48]:
print(f'Số lượng sinh viên thực tế: {len(set(diem_df["MaSV"]))}')

Số lượng sinh viên thực tế: 566


In [58]:
'''
Mục đích: Chuyển từ dạng 1 ký tự sang 2 ký tự trong NhomTo.
          Vì có trường hợp NhomTo là 2 nhưng khi mapping qua khgd thì không có (khgd chỉ có 02 không có 2).
'''
def transform_value(value):
    if len(value) == 1:
        return "0" + value
    else:
        return value

In [59]:
'''
Mục đích: Tách NHHK thành 2 cột năm học, học kỳ
          Sắp xếp theo thứ tự tăng dần về mặt thời gian:
            - So sánh năm học
            - Nếu bằng năm học thì so sánh học kỳ
'''
def split_nhhk(nhhk):
    return (nhhk[:4], nhhk[-1:])

In [62]:
# Apply hàm transform_value cho cột NhomTo trong diem_df
diem_df['NhomTo'] = diem_df['NhomTo'].apply(transform_value)
diem_df

,MaSV,MaMH,NHHK,NhomTo,DiemHP,DatHP
198,N14DCAT074,BAS1105,20141,07,8.7,1
199,N14DCAT074,BAS1106,20141,09,7.6,1
200,N14DCAT074,BAS1111,20141,05,6.6,1
201,N14DCAT074,BAS1201,20141,02,7.6,1
202,N14DCAT074,BAS1203,20141,02,6.5,1
...,...,...,...,...,...,...
89541,N18DCCN118,INT1461,20212,01,8.6,1
89542,N18DCCN118,INT14103,20221,01,8.2,1
89543,N18DCCN118,INT14104,20221,01,5.3,1
89544,N18DCCN118,INT1450,20221,01,6.8,1


In [34]:
khgd_df

,MaMH,NHHK,NhomTo,MaNV
0,BAS1115,20141,08,0211034
1,BAS1115,20141,13,0211034
2,BAS1115,20141,10,0211033
3,BAS1115,20141,06,0211033
4,BAS1115,20141,07,0211023
...,...,...,...,...
12451,ELE1317,20223,01,0211043
12452,ELE1433,20223,01,0211047
12453,BSA1241,20223,01,0211179
12454,SKD1103,20223,01,GV/N-20229


In [52]:
# Merge dữ liệu điểm với kế hoạch giảng dạy
# MaSV, MaMH, NHHK, NhomTo, DiemHP, DatHP, MaNV
df_saved = pd.merge(diem_df, khgd_df, how="left", on=["MaMH", "NHHK", "NhomTo"]).drop_duplicates(keep="first")

# Drop các record chứa 1 sinh viên, 1 NHHK, 1 nhóm tổ, 1 MaMH nhưng có 2 giảng viên dạy sau khi merge
# Các record khá ít, chỉ khoảng 10 record
# Ví dụ: 
# MSSV        MaMH      NHHK, NhomTo    DiemHP,  DatHP,    MaGV
# N17DCCN114, BAS1201, 20171, 01,       9.0,     1,       011111 <-- Lấy record này
# N17DCCN114, BAS1201, 20171, 01,       9.0,     1,       022222
# Chỉ giữ lại lần xuất hiện đầu tiên của record
df_saved = df_saved.drop_duplicates(subset=["MaSV", "MaMH", "NHHK", "NhomTo", "DiemHP", "DatHP"], keep="first")

# Tách cột NHHK ra 2 cột NamHoc và HocKy để tiện cho việc sắp xếp dữ liệu
df_saved["NamHoc"], df_saved["HocKy"] = zip(*df_saved["NHHK"].apply(split_nhhk))

# Lấy dữ liệu theo 2 cột mới
df_saved = df_saved[["MaSV", "MaMH", "NamHoc", "HocKy", "NhomTo", "DiemHP", "DatHP", "MaNV"]]

# Lọc bỏ các record mà DiemHP có chứa 1 hoặc nhiều các ký tự chữ cái (NaN = false)
# Sau đó đánh lại index từ 0 đến cuối
df_saved = df_saved[~df_saved["DiemHP"].str.contains('^[a-zA-Z]+$', na=False)].reset_index(drop=True)

In [53]:
df_saved.to_csv(f"{my_url}/data/data_mau.csv", index=True)